In [3]:
import mediapipe as mp
import cv2
import csv
import os
import numpy as np
import pandas as pd

In [4]:
mp_drawing = mp.solutions.drawing_utils
mp_holistic = mp.solutions.holistic

Creating Dataset

In [64]:
class_name = "Pataka"

In [65]:
cap = cv2.VideoCapture(0)

with mp_holistic.Holistic(min_detection_confidence=0.5,min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():
        ret,frame = cap.read()

        image=cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)
        image.flags.writeable=False
        
        results = holistic.process(image)

        image.flags.writeable=True
        image=cv2.cvtColor(image,cv2.COLOR_RGB2BGR)
        
        # 2. Right hand
        mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                                 )

        try:

            rhand = results.right_hand_landmarks.landmark
            rhand_row = list(np.array([[landmark.x,landmark.y,landmark.z,landmark.visibility]for landmark in rhand]).flatten())
            
            row = rhand_row
            row.insert(0,class_name)
            with open("Pataka.csv",mode='a',newline='') as f:
                csv_writer = csv.writer(f,delimiter=',',quotechar='"',quoting=csv.QUOTE_MINIMAL)
                csv_writer.writerow(row)

        except:
            pass

        cv2.imshow('raw webcam feed',image)

        if cv2.waitKey(10) & 0xFF ==ord('q'):
            break
        
cap.release()
cv2.destroyAllWindows()

Training

In [66]:
from sklearn.model_selection import train_test_split

In [67]:
df = pd.read_csv('Pataka.csv')
df.tail()

,class,x1,y1,z1,v1,x2,y2,z2,v2,x3,...,z19,v19,x20,y20,z20,v20,x21,y21,z21,v21
2579,Pataka,0.322937,0.793995,-0.000122,0.0,0.391784,0.780995,-0.059210,0.0,0.456118,...,-0.077881,0.0,0.323110,0.434589,-0.093488,0.0,0.328848,0.383105,-0.107246,0.0
2580,Pataka,0.323545,0.798884,-0.000117,0.0,0.392506,0.786779,-0.061575,0.0,0.455844,...,-0.071122,0.0,0.323211,0.433533,-0.084740,0.0,0.328982,0.384045,-0.096512,0.0
2581,Pataka,0.325910,0.804765,-0.000132,0.0,0.394022,0.790706,-0.060496,0.0,0.457629,...,-0.076009,0.0,0.328321,0.443011,-0.092214,0.0,0.334406,0.393730,-0.105662,0.0
2582,Pataka,0.329445,0.811656,-0.000109,0.0,0.402038,0.796336,-0.055525,0.0,0.466017,...,-0.082117,0.0,0.339561,0.448548,-0.101644,0.0,0.346865,0.397246,-0.118107,0.0
2583,Pataka,0.326847,0.813953,-0.000108,0.0,0.400441,0.800132,-0.057925,0.0,0.464254,...,-0.067407,0.0,0.334876,0.445957,-0.081634,0.0,0.341430,0.396766,-0.092330,0.0


In [68]:
X = df.drop('class',axis=1)
y = df['class']

In [69]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3,random_state=1234)

In [70]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

In [71]:
pipelines = {
    'lr':make_pipeline(StandardScaler(),LogisticRegression()),
    'rc':make_pipeline(StandardScaler(),RidgeClassifier()),
    'rf':make_pipeline(StandardScaler(),RandomForestClassifier()),
    'gb':make_pipeline(StandardScaler(),GradientBoostingClassifier())
}

In [72]:
list(pipelines.values())

[Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('logisticregression', LogisticRegression())]),
 Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('ridgeclassifier', RidgeClassifier())]),
 Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('randomforestclassifier', RandomForestClassifier())]),
 Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('gradientboostingclassifier', GradientBoostingClassifier())])]

In [73]:
fit_models = {}
for algo, pipeline in pipelines.items():
    model = pipeline.fit(X_train,y_train)
    fit_models[algo] = model

In [74]:
from sklearn.metrics import accuracy_score
import pickle

In [75]:
for algo, model in fit_models.items():
    yhat = model.predict(X_test)
    print(algo,accuracy_score(y_test,yhat))

lr 0.9922680412371134
rc 0.9922680412371134
rf 0.9948453608247423
gb 0.9974226804123711


In [76]:
fit_models['lr'].predict(X_test)

array(['Pataka', 'Pataka', 'Wrong', 'Wrong', 'Wrong', 'Pataka', 'Wrong',
       'Pataka', 'Pataka', 'Wrong', 'Wrong', 'Pataka', 'Wrong', 'Wrong',
       'Pataka', 'Wrong', 'Pataka', 'Wrong', 'Wrong', 'Wrong', 'Wrong',
       'Pataka', 'Wrong', 'Wrong', 'Pataka', 'Wrong', 'Wrong', 'Pataka',
       'Pataka', 'Wrong', 'Pataka', 'Wrong', 'Wrong', 'Pataka', 'Wrong',
       'Pataka', 'Wrong', 'Pataka', 'Wrong', 'Pataka', 'Wrong', 'Pataka',
       'Wrong', 'Pataka', 'Wrong', 'Wrong', 'Pataka', 'Wrong', 'Pataka',
       'Pataka', 'Wrong', 'Pataka', 'Pataka', 'Pataka', 'Wrong', 'Pataka',
       'Wrong', 'Wrong', 'Wrong', 'Wrong', 'Pataka', 'Pataka', 'Wrong',
       'Wrong', 'Wrong', 'Pataka', 'Wrong', 'Wrong', 'Pataka', 'Wrong',
       'Pataka', 'Wrong', 'Pataka', 'Wrong', 'Wrong', 'Wrong', 'Pataka',
       'Wrong', 'Wrong', 'Wrong', 'Wrong', 'Wrong', 'Wrong', 'Pataka',
       'Wrong', 'Wrong', 'Pataka', 'Wrong', 'Wrong', 'Wrong', 'Wrong',
       'Wrong', 'Wrong', 'Pataka', 'Wrong', 'Wrong', 'W

In [77]:
y_test

2141    Pataka
2322    Pataka
367      Wrong
658      Wrong
263      Wrong
         ...  
955      Wrong
1859    Pataka
643      Wrong
1976    Pataka
971      Wrong
Name: class, Length: 776, dtype: object

In [78]:
with open('Pataka.pkl','wb') as f:
    pickle.dump(fit_models['rf'],f)

Make Detection

In [11]:
import pickle
with open('Kartarimukh.pkl','rb') as f:
    model_body = pickle.load(f)

In [12]:
cap = cv2.VideoCapture(0)

with mp_holistic.Holistic(min_detection_confidence=0.5,min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():
        ret,frame = cap.read()

        image=cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)
        image.flags.writeable=False
        
        results = holistic.process(image)

        image.flags.writeable=True
        image=cv2.cvtColor(image,cv2.COLOR_RGB2BGR)

        # 2. Right hand
        mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                                 )

        try:
            
            #cv2.rectangle(image,(0,30),(len(image[0]), 60),(245,117,16),-1)

            rHand = results.right_hand_landmarks.landmark
            rHand_row = list(np.array([[landmark.x,landmark.y,landmark.z,landmark.visibility]for landmark in rHand]).flatten())   
            row = rHand_row

            
            # Hands
            X4 = pd.DataFrame([row])
            body_hands_class = model_body.predict(X4)[0]
            body_hands_prob = model_body.predict_proba(X4)[0]

            cv2.putText(image, "Pataka", (0, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
            cv2.putText(image, str(body_hands_prob[0]) , (0, 60), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
            
            
                

        except Exception as e:
            # print(e)
            pass

        cv2.imshow('raw webcam feed',image)

        if cv2.waitKey(10) & 0xFF ==ord('q'):
            break
        
cap.release()
cv2.destroyAllWindows()

c:\Users\glend\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
c:\Users\glend\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
c:\Users\glend\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
c:\Users\glend\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
c:\Users\glend\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler wa